In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split, Dataset, DataLoader, TensorDataset
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np
import copy
import random

print(torch.__version__)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
2.2.2


In [2]:
# find correct device
if torch.backends.mps.is_available():
    print("using mps")
    device = torch.device("mps")
elif torch.cuda.is_available():
    print("using cuda")
    device = torch.device("cuda")
else:
    print("using cpu")
    device = torch.device("cpu")

using mps


In [5]:
data_dict_1 = np.load("gt_clean_e1.npz")
data_dict_2 = np.load("gt_clean_e2.npz")
data_dict_3 = np.load("gt_clean_e3.npz")

In [6]:
ori32_1 = torch.tensor(data_dict_1["ori64"].transpose((2,1,0))[:,:,:].astype(np.float32))
sim32_1 = torch.tensor(data_dict_1["sim64"].transpose((2,1,0))[:,:,:].astype(np.float32))
ori32means_1 = torch.tensor(data_dict_1["ori64means"].transpose((2,1,0))[:,:,:].astype(np.float32))
sim32means_1 = torch.tensor(data_dict_1["sim64means"].transpose((2,1,0))[:,:,:].astype(np.float32))
ori32sigmas_1 = torch.tensor(data_dict_1["ori64sigmas"].transpose((2,1,0))[:,:,:].astype(np.float32))
sim32sigmas_1 = torch.tensor(data_dict_1["sim64sigmas"].transpose((2,1,0))[:,:,:].astype(np.float32))

In [7]:
ori32_2 = torch.tensor(data_dict_2["ori64"].transpose((2,1,0))[:,:,:].astype(np.float32))
sim32_2 = torch.tensor(data_dict_2["sim64"].transpose((2,1,0))[:,:,:].astype(np.float32))
ori32means_2 = torch.tensor(data_dict_2["ori64means"].transpose((2,1,0))[:,:,:].astype(np.float32))
sim32means_2 = torch.tensor(data_dict_2["sim64means"].transpose((2,1,0))[:,:,:].astype(np.float32))
ori32sigmas_2 = torch.tensor(data_dict_2["ori64sigmas"].transpose((2,1,0))[:,:,:].astype(np.float32))
sim32sigmas_2 = torch.tensor(data_dict_2["sim64sigmas"].transpose((2,1,0))[:,:,:].astype(np.float32))

In [9]:
ori32_3 = torch.tensor(data_dict_3["ori64"].transpose((2,1,0))[:,:,:].astype(np.float32))
sim32_3 = torch.tensor(data_dict_3["sim64"].transpose((2,1,0))[:,:,:].astype(np.float32))
ori32means_3 = torch.tensor(data_dict_3["ori64means"].transpose((2,1,0))[:,:,:].astype(np.float32))
sim32means_3 = torch.tensor(data_dict_3["sim64means"].transpose((2,1,0))[:,:,:].astype(np.float32))
ori32sigmas_3 = torch.tensor(data_dict_3["ori64sigmas"].transpose((2,1,0))[:,:,:].astype(np.float32))
sim32sigmas_3 = torch.tensor(data_dict_3["sim64sigmas"].transpose((2,1,0))[:,:,:].astype(np.float32))

In [10]:
# torch.nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 18, 9, padding="same")
        self.conv2 = nn.Conv1d(18, 18, 9, padding="same")
        self.conv3 = nn.Conv1d(18, 1, 1, padding="same")
        self.bn = nn.BatchNorm1d(18)
        self.dropout1 = nn.Dropout(0.10)
        self.dropout2 = nn.Dropout(0.5)

    def forward(self, x):
        x = self.conv1(x)
        x = F.sigmoid(x)
        
        x = self.conv2(x)
        x = self.bn(x)
        x = F.sigmoid(x)

        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.bn(x)
        x = F.sigmoid(x)

        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.bn(x)
        x = F.sigmoid(x)
        
        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.bn(x)
        x = F.sigmoid(x)

        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.bn(x)
        x = F.sigmoid(x)
        
        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.bn(x)
        x = F.sigmoid(x)

        x = self.dropout1(x)
        
        output = self.conv3(x)
        
        return output

In [12]:
# load model
model1 = Net()
model1.load_state_dict(torch.load("bd_clean_e1model.pt", weights_only=True))
model1.eval()
model2 = Net()
model2.load_state_dict(torch.load("bd_clean_e2model.pt", weights_only=True))
model2.eval()
model3 = Net()
model3.load_state_dict(torch.load("bd_clean_e3model.pt", weights_only=True))
model3.eval()

Net(
  (conv1): Conv1d(1, 18, kernel_size=(9,), stride=(1,), padding=same)
  (conv2): Conv1d(18, 18, kernel_size=(9,), stride=(1,), padding=same)
  (conv3): Conv1d(18, 1, kernel_size=(1,), stride=(1,), padding=same)
  (bn): BatchNorm1d(18, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
)

In [14]:
print("Corr dataset 1: ",torch.corrcoef(torch.stack((torch.flatten(ori32_1),torch.flatten(sim32_1)),dim=0))[1,0])
print("Corr dataset 2: ",torch.corrcoef(torch.stack((torch.flatten(ori32_2),torch.flatten(sim32_2)),dim=0))[1,0])
print("Corr dataset 3: ",torch.corrcoef(torch.stack((torch.flatten(ori32_3),torch.flatten(sim32_3)),dim=0))[1,0])

Corr dataset 1:  tensor(0.1288)
Corr dataset 2:  tensor(0.3237)
Corr dataset 3:  tensor(0.4450)


In [18]:
print("Corr dataset cleaned1: ",torch.corrcoef(torch.stack((torch.flatten(model1(ori32_1)),torch.flatten(sim32_1)),dim=0))[1,0])
print("Corr dataset cleaned2: ",torch.corrcoef(torch.stack((torch.flatten(model2(ori32_2)),torch.flatten(sim32_2)),dim=0))[1,0])
print("Corr dataset cleaned3: ",torch.corrcoef(torch.stack((torch.flatten(model3(ori32_3)),torch.flatten(sim32_3)),dim=0))[1,0])

Corr dataset cleaned1:  tensor(0.1608, grad_fn=<SelectBackward0>)
Corr dataset cleaned2:  tensor(0.3491, grad_fn=<SelectBackward0>)
Corr dataset cleaned3:  tensor(0.4649, grad_fn=<SelectBackward0>)


In [19]:
print("Corr dataset cleaned1 (Model3): ",torch.corrcoef(torch.stack((torch.flatten(model3(ori32_1)),torch.flatten(sim32_1)),dim=0))[1,0])
print("Corr dataset cleaned2 (Model3): ",torch.corrcoef(torch.stack((torch.flatten(model3(ori32_2)),torch.flatten(sim32_2)),dim=0))[1,0])
print("Corr dataset cleaned3 (Model3): ",torch.corrcoef(torch.stack((torch.flatten(model3(ori32_3)),torch.flatten(sim32_3)),dim=0))[1,0])

Corr dataset cleaned1 (Model3):  tensor(0.1365, grad_fn=<SelectBackward0>)
Corr dataset cleaned2 (Model3):  tensor(0.3371, grad_fn=<SelectBackward0>)
Corr dataset cleaned3 (Model3):  tensor(0.4649, grad_fn=<SelectBackward0>)


In [20]:
print("Corr dataset cleaned1 (Model2): ",torch.corrcoef(torch.stack((torch.flatten(model2(ori32_1)),torch.flatten(sim32_1)),dim=0))[1,0])
print("Corr dataset cleaned2 (Model2): ",torch.corrcoef(torch.stack((torch.flatten(model2(ori32_2)),torch.flatten(sim32_2)),dim=0))[1,0])
print("Corr dataset cleaned3 (Model2): ",torch.corrcoef(torch.stack((torch.flatten(model2(ori32_3)),torch.flatten(sim32_3)),dim=0))[1,0])

Corr dataset cleaned1 (Model2):  tensor(0.1490, grad_fn=<SelectBackward0>)
Corr dataset cleaned2 (Model2):  tensor(0.3491, grad_fn=<SelectBackward0>)
Corr dataset cleaned3 (Model2):  tensor(0.4446, grad_fn=<SelectBackward0>)


In [21]:
print("Corr dataset cleaned1 (Model1: ",torch.corrcoef(torch.stack((torch.flatten(model1(ori32_1)),torch.flatten(sim32_1)),dim=0))[1,0])
print("Corr dataset cleaned2 (Model1): ",torch.corrcoef(torch.stack((torch.flatten(model1(ori32_2)),torch.flatten(sim32_2)),dim=0))[1,0])
print("Corr dataset cleaned3 (Model1): ",torch.corrcoef(torch.stack((torch.flatten(model1(ori32_3)),torch.flatten(sim32_3)),dim=0))[1,0])

Corr dataset cleaned1 (Model1:  tensor(0.1608, grad_fn=<SelectBackward0>)
Corr dataset cleaned2 (Model1):  tensor(0.3177, grad_fn=<SelectBackward0>)
Corr dataset cleaned3 (Model1):  tensor(0.3993, grad_fn=<SelectBackward0>)
